In [1]:
import datetime
import joblib
from math import radians
from sml import cc_features

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pyplot
import warnings

In [3]:
import hopsworks
from sml import synthetic_data

import streamlit as st

import folium
from streamlit_folium import st_folium
import json

In [4]:
start_date = (datetime.datetime.now() - datetime.timedelta(hours=200)) 
end_date = (datetime.datetime.now()) 

synthetic_data.set_random_seed(12345)
credit_cards = [cc["cc_num"] for cc in synthetic_data.generate_list_credit_card_numbers()]
lat = 0
long = 0

warnings.filterwarnings("ignore")

In [5]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/72856
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
@st.cache(allow_output_mutation=True, suppress_st_warning=True)
def retrieve_dataset(fv, start_date, end_date):
    st.write(36 * "-")
    print_fancy_header('\n💾 Dataset Retrieving...')
    batch_data = fv.get_batch_data(start_time = start_date, end_time = end_date)
    batch_data.drop(["tid", "cc_num", "datetime"], axis = 1, inplace=True)
    return batch_data

In [7]:
@st.cache(suppress_st_warning=True, allow_output_mutation=True)
def get_feature_view():
    fv = fs.get_feature_view("cc_trans_fraud", 1)
    return fv

In [8]:
@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def get_model(project = project):
    mr = project.get_model_registry()
    model = mr.get_model("cc_fraud", version = 1)
    model_dir = model.download()
    return joblib.load(model_dir + "/cc_fraud_model.pkl")

In [9]:
def explore_data(batch_data):
    st.write(36 * "-")
    print_fancy_header('\n👁 Data Exploration...')
    labels = ["Suspected of Fraud", "Not Suspected of Fraud"]
    unique, counts = np.unique(batch_data.fraud.values, return_counts=True)
    values = counts.tolist()

    def plot_pie(values, labels):
        fig = px.pie(values=values, names=labels, title='Distribution of predicted fraud transactions')
        return fig

    fig1 = plot_pie(values, labels)
    st.plotly_chart(fig1)

In [10]:
def print_fancy_header(text, font_size=24):
    res = f'<span style="color:#ff5f27; font-size: {font_size}px;">{text}</span>'
    st.markdown(res, unsafe_allow_html=True)

In [11]:
def transform_preds(predictions):
    return ['Fraud' if pred == 1 else 'Not Fraud' for pred in predictions]

In [12]:
progress_bar = st.sidebar.header('⚙️ Working Progress')
progress_bar = st.sidebar.progress(0)
st.title('🆘 Fraud transactions detection 🆘')

2023-07-14 00:29:32.626 
  command:

    streamlit run /Users/xianyao/myenv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [13]:
st.write(36 * "-")
print_fancy_header('\n📡 Connecting to Hopsworks Feature Store...')

In [14]:
st.write(36 * "-")
print_fancy_header('\n🤖 Connecting to Model Registry on Hopsworks...')
model = get_model(project)
st.write(model)
st.write("✅ Connected!")

InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `get_model()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function get_model at 0x7f9ab1640a60>
```

Please see the `hash_funcs` [documentation](https://docs.streamlit.io/library/advanced-features/caching#the-hash_fun

In [15]:
progress_bar.progress(40)

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())

In [16]:
st.write(36 * "-")
print_fancy_header('\n✨ Fetch batch data and predict')
fv = get_feature_view()

InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `get_feature_view()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function get_feature_view at 0x7f9ab1640af0>
```

Please see the `hash_funcs` [documentation](https://docs.streamlit.io/library/advanced-features/cachin

In [17]:
if st.button('📊 Make a prediction'):
    batch_data = retrieve_dataset(fv, start_date, end_date)
    st.write("✅ Retrieved!")
    progress_bar.progress(55)
    predictions = model.predict(batch_data)
    predictions = transform_preds(predictions)
    batch_data_to_explore = batch_data.copy()
    batch_data_to_explore['fraud'] = predictions
    explore_data(batch_data_to_explore)

In [18]:
st.button("Re-run")

False